Notebook to retrieve affordable housing data from CoStar (sample)

<b>Author</b>: Phu Dang

<b>Date</b>: November 24, 2023

In [39]:
import pandas as pd 
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

In [229]:
pd.set_option('display.max_rows', None)

In [267]:
# Test / Demo

df = pd.read_clipboard(sep=r'^', skip_blank_lines=False)

In [268]:
data = df.copy()

In [269]:
def viewAll(status=False):

    if status:
        pd.set_option('display.max_rows', None)
    else:
        pd.set_option('display.max_rows', 11)
        
    return None

In [270]:
data.iloc[range(15), :]

,3501 1st Ave
0,3501 1st Avenue
1,C
2,22
3,"16,354"
4,Apartments
5,Hillcrest
6,San Diego
7,0.31
8,1960
9,1


In [271]:
viewAll(False)

In [272]:
data

,3501 1st Ave
0,3501 1st Avenue
1,C
2,22
3,"16,354"
4,Apartments
...,...
1115,0.9
1116,Affordable
1117,Rent Restricted
1118,0.93


In [273]:
# Operation: Make initial column name the first row, rename column as 'data'
# Reason: The first property's address is pasted as the column name (the
#   immediate dataset after pasting only has 1 column)

if data.columns[0] != 'data':
    firstPropertyAddress = data.columns[0]
    topRow = pd.DataFrame(data={'data': [firstPropertyAddress]})
    data.rename(columns={firstPropertyAddress: 'data'}, inplace=True)
    data = pd.concat([topRow, data], axis=0).reset_index(drop=True)

In [274]:
# sanity check

for i in range(5):
    if i == 0:
        pass
    else:
        print('Hello')
    
    print('HA')

HA
Hello
HA
Hello
HA
Hello
HA
Hello
HA


In [275]:
addressPattern = r'^\d+.+[a-zA-Z]{1}$'
classPattern = r'^[A-Z]{1}$'
builtRenovPattern = r'(^[0-9]{4}$)|(^[0-9]{4}/{1}[0-9]{4}$)'
secondaryTypes = ['Manufactured Housing/Mobile Home Park', 'Dormitory', \
    'Apartments']
cities = ['del mar', 'la jolla', 'pacific beach', 'san diego']
rentTypes = ['market/affordable', 'market', 'affordable']
affordableTypes = ['rent subsidized', 'rent stabilized', 'rent restricted', \
    'rent controlled', 'affordable units']
propWORentType = ['3357 Apache Ave',
                    '2179 Avenida De La Playa',
                    '941 Camino Del Mar',
                    '1765 Chalcedony St',
                    '2596 Chalcedony St',
                    '3867-3869 Clairemont Dr',
                    '4828 Clairemont Dr',
                    '2135 Coast Blvd',
                    '1671-1675 Diamond St',
                    '826 Emerald St',
                    '918 Felspar St',
                    '2142 Felspar St',
                    '1425 Grand Ave',
                    '4274-4276 Gresham St',
                    '2732-2736 Hornblend St',
                    '4330 Kendall',
                    '7696-7698 Kiwi St',
                    '6850 Mission Gorge Rd',
                    '377-385 Nautilus St',
                    '1217 Oliver Ave',
                    '6232-6236 Osler St',
                    '343 Playa Del Sur',
                    '3701 Promontory St',
                    '4044 Promontory St',
                    '725 Rockaway Ct',
                    '961 Thomas Ave',
                    '1052 Turquoise St',
                    '702-708 Whiting Ct',
                    '7634-7676 Clairemont',
                    '3410 Clairemont Dr',
                    '1033 Diamond',
                    '1950 Emerald St',
                    '539 Genter St',
                    '5550 Kearny Mesa Rd',
                    '4330 Kendall St',
                    '1719 Oliver Ave',
                    '922-926 Reed Ave',
                    '10288 Wateridge Cir']

In [276]:
data[:15]

,data
0,3501 1st Ave
1,3501 1st Avenue
2,C
3,22
4,"16,354"
...,...
10,1
11,2
12,2.3
13,Affordable


In [284]:
def attributeCheck(datum, column, currIdx):

    output = None

    if column == 'Address':
        if re.findall(addressPattern, datum):
            output = True
        else: output = False

    elif column == 'Building Name':
        prev_d = data['data'][currIdx-1]
        if re.findall(r'[A-Za-z]{2,}', datum) and \
            re.findall(addressPattern, prev_d):
            output = True
        else: output = False

    elif column == 'Class':
        if re.findall(classPattern, datum): 
            output = True 
        else: output = False

    elif column == 'Units':
        if list(finalDf['Address'])[-1] == '2135 Coast Blvd':
            output = False
        else: output = True

    elif column == 'RBA/GLA':
        if list(finalDf['Address'])[-1] == '10770-10771 Black Mountain Rd':
            output = False
        else: output = True 

    elif column == 'Secondary Type':
        if datum in secondaryTypes:
            output = True
        else: output = False
    
    elif column == 'Submarket':
        prev_d = data['data'][currIdx-1]
        if (prev_d in secondaryTypes) or (len(prev_d) == 5):
            output = True
        else: output = False

    elif column == 'City':
        if datum.lower() in cities:
            output = True
        else: output = False
    
    elif column == 'Land(AC)':
        prev_d = data['data'][currIdx-1]
        if ('.' in datum) and (prev_d.lower() in cities):
            output = True
        else: output = False
    
    elif column == 'Built/Renov':
        if re.findall(builtRenovPattern, datum):
            output = True
        else: output = False
    
    elif column == 'Total Buildings':
        prev_d = data['data'][currIdx-1]
        if (re.findall(builtRenovPattern, prev_d)) or ('.' in prev_d):
            output = True
        else: output = False 

    elif column == 'Stories':
        next_d = data['data'][currIdx+1].lower()
        if ('.' not in datum) and ((next_d in rentTypes) or ('.' in next_d)):
            output = True
        else: output = False
    
    elif column == 'Vacancy %':
        next_d = data['data'][currIdx+1]
        if (next_d.lower() in rentTypes) and ('.' in datum):
            output = True
        elif ('.' in datum):
            output = True
        else: output = False

    elif column == 'Rent Type':
        output = True if datum.lower() in rentTypes else False
    
    elif column == 'Affordable Type':
        output = True if datum.lower() in affordableTypes else False
    
    elif column == 'Parking Spaces/Unit':
        prev_d = data['data'][currIdx-1].lower()
        if ('.' in datum) and ((prev_d in rentTypes+affordableTypes) or \
            (len(prev_d) == 1)):
            output = True
        else: output = False
    
    else:
        next_idx = min(len(df),currIdx+1)
        if next_idx == len(df): # special case for the last entry
            output = True
        else:
            next_d = data['data'][next_idx]
            if re.findall(addressPattern, next_d) and \
                not re.findall(addressPattern, datum):
                output = True
            else: output = False

    return output 

In [285]:
# Create final dataframe

colNames = ['Address', 'Building Name', 'Class', 'Units', 'RBA/GLA', \
    'Secondary Type', 'Submarket', 'City', 'Land(AC)', 'Built/Renov', \
    'Total Buildings', 'Stories', 'Vacancy %', 'Rent Type', 'Affordable Type', \
    'Parking Spaces/Unit', 'Parking Spaces']
finalDf = pd.DataFrame(columns=colNames)

In [286]:
# Begin iterating over data, identify null attributes, populate final dataframe

prev = (None, None)
i = 0
while i < len(data['data']):
    
    # if finalDf.shape[0] == 2:
    #     break

    for column in colNames:
        d = data['data'][i]
        status = attributeCheck(d, column, currIdx=i)
        print(f"{d}, {column}, {status}")
        if status:
            if column == 'Address':
                finalDf.loc[finalDf.shape[0], 'Address'] = d
            else: 
                finalDf.loc[finalDf.shape[0]-1, column] = d 
            i += 1
        else: 
            finalDf.loc[finalDf.shape[0]-1, column] = np.NaN 

3501 1st Ave, Address, True
3501 1st Avenue, Building Name, True
C, Class, True
22, Units, True
16,354, RBA/GLA, True
Apartments, Secondary Type, True
Hillcrest, Submarket, True
San Diego, City, True
0.31, Land(AC), True
1960, Built/Renov, True
1, Total Buildings, True
2, Stories, True
2.3, Vacancy %, True
Affordable, Rent Type, True
Rent Restricted, Affordable Type, True
0.73, Parking Spaces/Unit, True
16, Parking Spaces, True
3534 5th Ave, Address, True
Strauss on Fifth, Building Name, True
B, Class, True
141, Units, True
261,900, RBA/GLA, True
Apartments, Secondary Type, True
Hillcrest, Submarket, True
San Diego, City, True
0.98, Land(AC), True
2019, Built/Renov, True
1, Total Buildings, True
6, Stories, True
0.8, Vacancy %, True
Market/Affordable, Rent Type, True
Affordable Units, Affordable Type, True
3754 5th Ave, Parking Spaces/Unit, False
3754 5th Ave, Parking Spaces, False
3754 5th Ave, Address, True
Hillcrest Inn, Building Name, True
C, Class, True
46, Units, True
47,000, RBA

In [287]:
finalDf

,Address,Building Name,Class,Units,RBA/GLA,Secondary Type,Submarket,City,Land(AC),Built/Renov,Total Buildings,Stories,Vacancy %,Rent Type,Affordable Type,Parking Spaces/Unit,Parking Spaces
0,3501 1st Ave,3501 1st Avenue,C,22,"16,354",Apartments,Hillcrest,San Diego,0.31,1960,1,2,2.3,Affordable,Rent Restricted,0.73,16
1,3534 5th Ave,Strauss on Fifth,B,141,"261,900",Apartments,Hillcrest,San Diego,0.98,2019,1,6,0.8,Market/Affordable,Affordable Units,NaN,NaN
2,3754 5th Ave,Hillcrest Inn,C,46,"47,000",Apartments,Hillcrest,San Diego,0.16,1988,1,3,4.5,Affordable,Affordable Units,NaN,NaN
3,3705 10th Ave,NaN,C,5,"7,460",Apartments,Uptown San Diego,San Diego,0.22,2001,1,1,2.3,Affordable,Rent Subsidized,1.60,8
4,2104-2106 Abbott St,NaN,C,2,"1,944",Apartments,Ocean Beach,San Diego,0.11,1950,2,1,4.3,Market/Affordable,Affordable Units,NaN,NaN
5,4122-4128 Bannock Ave,NaN,C,4,"3,032",Apartments,Clairemont,San Diego,0.30,1954,2,1,2.9,Affordable,Affordable Units,NaN,6
6,7720 Belden St,Village North Senior Garden Apartments,C,120,"61,468",Apartments,Clairemont,San Diego,2.36,1985,7,2,0.9,Affordable,Rent Restricted,0.75,90
7,4503 Bond St,NaN,NaN,2,"1,720",Apartments,Pacific Beach,San Diego,0.13,1953/2020,2,1,3.3,Market/Affordable,Rent Controlled,NaN,4
8,12730 Briarcrest Pl,Windwood Village,B,92,"91,879",Apartments,Carmel Valley,San Diego,4.38,2003,11,2,0.0,Affordable,Rent Restricted,3.26,300
9,3884 Caminito Aguilar,The Stratton Apartments,C,312,"286,925",Apartments,Clairemont,San Diego,10.62,1973,17,3,3.1,Market/Affordable,Rent Restricted,1.92,600


In [288]:
viewAll(True) # print full dataframe

In [289]:
finalDf

,Address,Building Name,Class,Units,RBA/GLA,Secondary Type,Submarket,City,Land(AC),Built/Renov,Total Buildings,Stories,Vacancy %,Rent Type,Affordable Type,Parking Spaces/Unit,Parking Spaces
0,3501 1st Ave,3501 1st Avenue,C,22,"16,354",Apartments,Hillcrest,San Diego,0.31,1960,1,2,2.3,Affordable,Rent Restricted,0.73,16
1,3534 5th Ave,Strauss on Fifth,B,141,"261,900",Apartments,Hillcrest,San Diego,0.98,2019,1,6,0.8,Market/Affordable,Affordable Units,NaN,NaN
2,3754 5th Ave,Hillcrest Inn,C,46,"47,000",Apartments,Hillcrest,San Diego,0.16,1988,1,3,4.5,Affordable,Affordable Units,NaN,NaN
3,3705 10th Ave,NaN,C,5,"7,460",Apartments,Uptown San Diego,San Diego,0.22,2001,1,1,2.3,Affordable,Rent Subsidized,1.60,8
4,2104-2106 Abbott St,NaN,C,2,"1,944",Apartments,Ocean Beach,San Diego,0.11,1950,2,1,4.3,Market/Affordable,Affordable Units,NaN,NaN
5,4122-4128 Bannock Ave,NaN,C,4,"3,032",Apartments,Clairemont,San Diego,0.30,1954,2,1,2.9,Affordable,Affordable Units,NaN,6
6,7720 Belden St,Village North Senior Garden Apartments,C,120,"61,468",Apartments,Clairemont,San Diego,2.36,1985,7,2,0.9,Affordable,Rent Restricted,0.75,90
7,4503 Bond St,NaN,NaN,2,"1,720",Apartments,Pacific Beach,San Diego,0.13,1953/2020,2,1,3.3,Market/Affordable,Rent Controlled,NaN,4
8,12730 Briarcrest Pl,Windwood Village,B,92,"91,879",Apartments,Carmel Valley,San Diego,4.38,2003,11,2,0.0,Affordable,Rent Restricted,3.26,300
9,3884 Caminito Aguilar,The Stratton Apartments,C,312,"286,925",Apartments,Clairemont,San Diego,10.62,1973,17,3,3.1,Market/Affordable,Rent Restricted,1.92,600


In [291]:
# regular expression tester

test = '1992/'
pattern = r'(^[0-9]{4}$)|(^[0-9]{4}/{1}[0-9]{4}$)'
result = re.findall(pattern, test)
result

[]

##### Save dataframe samples

In [227]:
# wp = "C:/Users/phuro/UCSD/ULI/H2H/ULI-UCSD_H2H/sample/original.csv"

# df.to_csv(wp)

In [228]:
# wp =  "C:/Users/phuro/UCSD/ULI/H2H/ULI-UCSD_H2H/sample/processed.csv"

# finalDf.to_csv(wp)

##### Save final dataframes (market/affordable and affordable)

In [295]:
# wp = "C:/Users/phuro/UCSD/ULI/H2H/ULI-UCSD_H2H/market-affordable/original.csv"

# df.to_csv(wp)

In [296]:
# wp =  "C:/Users/phuro/UCSD/ULI/H2H/ULI-UCSD_H2H/market-affordable/processed.csv"

# finalDf.to_csv(wp)

##### Unused code archive

In [ ]:
# prev = (None, None)
# for i in range(len(data['data'])):
#     d = data['data'][i]

#     # Check if d is a property address
#     addressPattern = r'^\d+.+[a-zA-Z]{1}$'
#     result = re.findall(addressPattern, d)
#     if result:
#         if prev[1] == 'Address': 
#             pass
#         else:
#             finalDf.loc[finalDf.shape[0], 'Address'] = d
#             prev = (d, 'Address')
#             continue

#     # Check if d is a building name
#     classPattern = r'^[A-Z]{1}$'
#     next_d = data['data'][i+1]
#     if (prev[1] == 'Address') and (re.findall(classPattern, next_d)):
#         finalDf.loc[finalDf.shape[0]-1, 'Building Name'] = d
#         prev = (d, 'Building Name')
#         continue
#     elif (prev[1] == 'Address') and (re.findall(classPattern, d)):
#         finalDf.loc[finalDf.shape[0]-1, 'Building Name'] = np.NaN
#         finalDf.loc[finalDf.shape[0]-1, 'Class'] = d
#         prev = (d, 'Class')
#         continue

#     # Check if d is a class